In [3]:
bases <- c("x", "y")
coev_pair <- c("yy", "xx")

In [6]:
Q <- matrix(c('-u', 'u', 'u', '-u'), nrow = 2, ncol = 2)
colnames(Q) <- bases
rownames(Q) <- bases
Q_df <- as.data.frame(Q)
print(Q_df)

   x  y
x -u  u
y  u -u


In [7]:
# three site coevolution model
coev_factor <- 'f'
Qco3 <- matrix(nrow = 8, ncol = 8)
base_tri <- expand.grid(bases, bases, bases)
base_tri <- paste0(base_tri[, 1], base_tri[, 2], base_tri[, 3])
colnames(Qco3) <- base_tri
rownames(Qco3) <- base_tri
siteAB_coev_pair <- c("yy", "xx")
siteBC_coev_pair <- c("yy", "xx")
for (i in 1:nrow(Qco3)) {
    for (j in 1:ncol(Qco3)) {
        # skip if there is no change
        if (i == j) {
            next
        }
        siteA_change <- c(strsplit(base_tri[i], "")[[1]][1], strsplit(base_tri[j], "")[[1]][1])
        siteB_change <- c(strsplit(base_tri[i], "")[[1]][2], strsplit(base_tri[j], "")[[1]][2])
        siteC_change <- c(strsplit(base_tri[i], "")[[1]][3], strsplit(base_tri[j], "")[[1]][3])

        siteA_is_change <- as.numeric(siteA_change[1] != siteA_change[2])
        siteB_is_change <- as.numeric(siteB_change[1] != siteB_change[2])
        siteC_is_change <- as.numeric(siteC_change[1] != siteC_change[2])
        # treat changes in >2 sites simultaneously as impossible to happen
        if (siteA_is_change + siteB_is_change + siteC_is_change >= 2) {
            Qco3[i, j] <- '0'
            next
        }
        # consider siteA change, this site is changing in independantly from other sites
        if (siteA_is_change == 1) {
            Qco3[i, j] <- Q[siteA_change[1], siteA_change[2]]
            next
        }
        # consider siteB change, this site is dependant on the second site
        if (siteB_is_change == 1) {
            # first set the change as if siteB is changing independantly
            Qco3[i, j] <- Q[siteB_change[1], siteB_change[2]]
            # get the bases that siteA and B are changing from
            baseAB <- paste0(siteA_change[1], siteB_change[1])
            if (baseAB %in% siteAB_coev_pair) {
                Qco3[i, j] <- paste0(Qco3[i, j], '/', coev_factor)
            }
            # get the bases that siteA and B are changing into
            baseAB <- paste0(siteA_change[2], siteB_change[2])
            if (baseAB %in% siteAB_coev_pair) {
                Qco3[i, j] <- paste0(Qco3[i, j], coev_factor)
            }
            next
        }
        # consider siteC change, this site is dependant on the second site
        if (siteC_is_change == 1) {
            # first set the change as if siteC is changing independantly
            Qco3[i, j] <- Q[siteC_change[1], siteC_change[2]]
            # get the bases that siteB and C are changing from
            baseBC <- paste0(siteB_change[1], siteC_change[1])
            if (baseBC %in% siteBC_coev_pair) {
                Qco3[i, j] <- paste0(Qco3[i, j], '/', coev_factor)
            }
            # get the bases that siteB and C are changing into
            baseBC <- paste0(siteB_change[2], siteC_change[2])
            if (baseBC %in% siteBC_coev_pair) {
                Qco3[i, j] <- paste0(Qco3[i, j],  coev_factor)
            }
            next
        }
    }
}

In [8]:
Qco3

,xxx,yxx,xyx,yyx,xxy,yxy,xyy,yyy
xxx,NA,u,u/f,0,u/f,0,0,0
yxx,u,NA,0,uf,0,u/f,0,0
xyx,uf,0,NA,u,0,0,uf,0
yyx,0,u/f,u,NA,0,0,0,uf
xxy,uf,0,0,0,NA,u,u/f,0
yxy,0,uf,0,0,u,NA,0,uf
xyy,0,0,u/f,0,uf,0,NA,u
yyy,0,0,0,u/f,0,u/f,u,NA
